## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-25-06-19-40 +0000,nypost,Prince Harry fires back at Trump over NATO cri...,https://nypost.com/2026/01/25/us-news/prince-h...
1,2026-01-25-06-13-00 +0000,wsj,"In Battle for Evidence, Minnesota Sues Feds Ov...",https://www.wsj.com/us-news/in-battle-for-evid...
2,2026-01-25-06-10-38 +0000,wapo,Live updates: Fatal shooting in Minneapolis re...,https://www.washingtonpost.com/nation/2026/01/...
3,2026-01-25-06-10-38 +0000,wapo,Responding to a lawsuit filed Saturday by Minn...,https://www.washingtonpost.com
4,2026-01-25-06-02-10 +0000,nypost,Florida Rep. Maxwell Frost says he was punched...,https://nypost.com/2026/01/25/us-news/florida-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2434/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,minneapolis,22
3,trump,22
17,shooting,19
144,ice,16
61,alex,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
73,2026-01-24-22-55-08 +0000,nypost,Chilling new video reveals new angle of fed sh...,https://nypost.com/2026/01/24/us-news/chilling...,100
43,2026-01-25-01-50-20 +0000,nypost,Chaotic moment federal officer reached for gun...,https://nypost.com/2026/01/24/us-news/moment-f...,89
69,2026-01-24-23-17-59 +0000,nypost,"Who is Alex Jeffrey Pretti, the anti-ICE prote...",https://nypost.com/us-news/who-is-alex-jeffrey...,84
15,2026-01-25-04-41-39 +0000,nyt,"In Court Filings, Witnesses Describe Fatal Min...",https://www.nytimes.com/2026/01/24/us/witnesse...,73
18,2026-01-25-04-19-46 +0000,bbc,What we know about fatal shooting of Alex Pret...,https://www.bbc.com/news/articles/c20zjyxep99o...,73


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
73,100,2026-01-24-22-55-08 +0000,nypost,Chilling new video reveals new angle of fed sh...,https://nypost.com/2026/01/24/us-news/chilling...
127,66,2026-01-24-14-21-45 +0000,cbc,Trump threatens 100% tariffs on all Canadian g...,https://www.cbc.ca/news/politics/trump-tariff-...
92,47,2026-01-24-20-47-10 +0000,nypost,"Trilateral talks between US, Russia and Ukrain...",https://nypost.com/2026/01/24/world-news/trila...
14,39,2026-01-25-04-44-43 +0000,nypost,Rock climber Alex Honnold reaches top of Taipe...,https://nypost.com/2026/01/24/lifestyle/rock-c...
145,29,2026-01-24-12-48-31 +0000,nypost,Trump tells The Post he’s skipping the Super B...,https://nypost.com/2026/01/24/us-news/trump-te...
7,27,2026-01-25-05-17-00 +0000,wsj,Senate Democrats signaled they are willing to ...,https://www.wsj.com/politics/policy/senate-dem...
13,27,2026-01-25-04-45-07 +0000,nypost,Sen. Ted Cruz has snarky response after facin...,https://nypost.com/2026/01/24/us-news/sen-ted-...
5,26,2026-01-25-05-45-50 +0000,nypost,"Australian boy, 12, dies following shark attac...",https://nypost.com/2026/01/25/world-news/austr...
76,26,2026-01-24-22-30-27 +0000,nypost,Feds were arresting Ecuadorian illegal alien J...,https://nypost.com/2026/01/24/us-news/feds-wer...
117,26,2026-01-24-17-06-00 +0000,wsj,China’s Xi Places His Top General Under Invest...,https://www.wsj.com/world/china/chinas-xi-plac...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
